In [3]:
import sklearn as sk
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import MeanShift, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction import FeatureHasher
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import AgglomerativeClustering

In [4]:
# List of CSV file names
csv_files = ['Books.csv']

# Function to read and clean CSV files
def clean_csv_files(file_list):
    for file in file_list:
        # Check if the file exists
        if os.path.exists(file):
            # Read the CSV file
            df = pd.read_csv(file)
            # Drop rows with any null values
            df_cleaned = df.dropna()
            # Create a new file name with '_cleaned' appended
            new_file_name = os.path.splitext(file)[0] + '_cleaned.csv'
            # Save the cleaned dataframe to a new CSV file
            df_cleaned.to_csv(new_file_name, index=False)
            print(f'Cleaned {file} and saved the new file as {new_file_name}.')
        else:
            print(f'File {file} does not exist.')

# Call the function
clean_csv_files(csv_files)

File Books.csv does not exist.


In [5]:
def clean_and_trim_csv_files(file_list):
    for file in file_list:
        # Check if the file exists
        if os.path.exists(file):
            # Read the CSV file
            df = pd.read_csv(file)
            # Drop the last 2 columns
            df_trimmed = df.iloc[:, :-6]
            # Drop rows with any null values
            df_cleaned = df_trimmed.dropna()
            # Create a new file name with '_cleaned' appended
            new_file_name = os.path.splitext(file)[0] + '_cleaned.csv'
            # Save the cleaned dataframe to a new CSV file
            df_cleaned.to_csv(new_file_name, index=False)
            print(f'Processed {file}, dropped the last 2 columns, removed nulls, and saved as {new_file_name}.')
        else:
            print(f'File {file} does not exist.')

# List of CSV file names
csv_files = ['books.csv']

# Call the new function
clean_and_trim_csv_files(csv_files)

File books.csv does not exist.


In [6]:
# Function to read CSV, compute average ratings, merge datasets and identify non-matching ISBNs
def combine_datasets(ratings_file, books_file):
    # Read the datasets
    ratings_df = pd.read_csv(ratings_file)
    books_df = pd.read_csv(books_file)
    
    # Convert ISBNs to strings to ensure matching works correctly
    ratings_df['ISBN'] = ratings_df['ISBN'].astype(str)
    books_df['ISBN'] = books_df['ISBN'].astype(str)
    
    # Group the ratings by ISBN and calculate the average rating
    average_ratings = ratings_df.groupby('ISBN')['Book-Rating'].mean().reset_index()
    
    # Merge the datasets on ISBN
    merged_df = pd.merge(average_ratings, books_df, on='ISBN', how='outer', indicator=True)
    
    # Separate out the non-matching entries
    non_matching = merged_df[merged_df['_merge'] != 'both']
    
    # Check and print non-matching ISBNs
    if not non_matching.empty:
        print("Non-matching ISBNs:")
        print(non_matching[['ISBN', '_merge']])
    else:
        print("All ISBNs match between the datasets.")
    
    # Return only the rows that matched both
    return merged_df[merged_df['_merge'] == 'both'].drop(columns=['_merge'])

# Replace 'ratings.csv' and 'books.csv' with your actual file paths
combined_df = combine_datasets(r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\ratings_cleaned.csv', r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\books_cleaned.csv')

# You can now work with 'combined_df' which contains the merged data
print(combined_df.head())  # This line is just to show the first few entries of the combined dataframe

C:\Users\tyler\AppData\Local\Temp\ipykernel_7712\2049580182.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(books_file)


Non-matching ISBNs:
                 ISBN      _merge
0          904492401X   left_only
1         #069580216X   left_only
2            #6612432   left_only
3       (THEWINDMILLP   left_only
4          )416195113   left_only
...               ...         ...
341233     1874166633  right_only
341234      130897930  right_only
341235     020130998X  right_only
341236     2268032019  right_only
341237     3442150663  right_only

[71071 rows x 2 columns]
          ISBN  Book-Rating  \
52  000104687X          6.0   
53  000104799X          7.5   
56  000123207X          0.0   
57  000160418X          3.5   
59  000171421X          0.0   

                                           Book-Title     Book-Author  \
52  T.S. Eliot Reading \The Wasteland\" and Other ...      T.S. Eliot   
53                                        Monk's-hood    Ellis Peters   
56                        Paddington's Birthday Party    Michael Bond   
57                     The Clue in the Crumbling Wall   Carolyn Kee

In [7]:
def clean_datasets(ratings_file, books_file):
    # Read the datasets
    ratings_df = pd.read_csv(ratings_file)
    books_df = pd.read_csv(books_file)
    
    # Convert ISBNs to strings
    ratings_df['ISBN'] = ratings_df['ISBN'].astype(str)
    books_df['ISBN'] = books_df['ISBN'].astype(str)
    
    # Group the ratings by ISBN and calculate the average rating
    average_ratings = ratings_df.groupby('ISBN')['Book-Rating'].mean().reset_index()
    
    # Merge the datasets on ISBN with an indicator
    merged_df = pd.merge(average_ratings, books_df, on='ISBN', how='outer', indicator=True)
    
    # Remove entries that didn't match in the books dataset ('left_only')
    clean_merged_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=['_merge'])
    
    # Return the cleaned merged DataFrame
    return clean_merged_df

# Use the function to clean the datasets
# Replace 'ratings.csv' and 'books.csv' with the actual paths to your files
cleaned_combined_df = clean_datasets(r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\ratings_cleaned.csv', r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\books_cleaned.csv')

# Display the first few entries of the cleaned dataframe
cleaned_combined_df.head()


C:\Users\tyler\AppData\Local\Temp\ipykernel_7712\137031853.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(books_file)


,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
52,000104687X,6.0,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993,HarperCollins Publishers
53,000104799X,7.5,Monk's-hood,Ellis Peters,1994,HarperCollins Publishers
56,000123207X,0.0,Paddington's Birthday Party,Michael Bond,1942,HarperCollins Publishers
57,000160418X,3.5,The Clue in the Crumbling Wall,Carolyn Keene,1984,HarperCollins Publishers
59,000171421X,0.0,It's Not Easy Being a Bunny (A Beginner Book),Marilyn Sadler,1984,HarperCollins Publishers


In [17]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

# Function to sample a percentage of the data
def sample_data(df, percentage=50):
    return df.sample(frac=percentage/100)

# Set the percentage of data to use
data_percentage = 10  # Change this value between 0 and 100 as needed

# Sample the data
sampled_df = sample_data(cleaned_combined_df, percentage=data_percentage)

# Preprocessing steps
sampled_df['Year-Of-Publication'] = pd.to_numeric(sampled_df['Year-Of-Publication'], errors='coerce')
sampled_df = sampled_df.dropna(subset=['Year-Of-Publication'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(sampled_df[['Year-Of-Publication', 'Book-Rating']])
hasher = FeatureHasher(n_features=100, input_type='string')
hashed_features = hasher.transform(sampled_df['Book-Author'].apply(lambda x: [x]).tolist()).toarray()
X = np.hstack((scaled_features, hashed_features))

# Range of k values to test
k_values = range(90, 110)  # Test from 2 to 9 clusters

# Dictionary to store silhouette scores for each k
silhouette_scores = {}

for k in k_values:
    mbk = MiniBatchKMeans(n_clusters=k, batch_size=100, verbose=0)
    mbk_labels = mbk.fit_predict(X)
    silhouette_score_k = silhouette_score(X, mbk_labels)
    silhouette_scores[k] = silhouette_score_k
    print(f'k={k}, Silhouette Score: {silhouette_score_k}')

# Output the best k value
best_k = max(silhouette_scores, key=silhouette_scores.get)
print(f'Best k value: {best_k} with a silhouette score of {silhouette_scores[best_k]}')


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


k=90, Silhouette Score: 0.22021024048372773


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


k=91, Silhouette Score: 0.214690987911686


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


...

k=107, Silhouette Score: 0.2555457305231726


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


k=108, Silhouette Score: 0.24463619929197


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


k=109, Silhouette Score: 0.2537133089449231
Best k value: 105 with a silhouette score of 0.2570457705990885


In [19]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

# Function to sample a percentage of the data
def sample_data(df, percentage=50):
    return df.sample(frac=percentage/100)

# Set the percentage of data to use
data_percentage = 10  # Change this value between 0 and 100 as needed

# Sample the data
sampled_df = sample_data(cleaned_combined_df, percentage=data_percentage)

# Preprocessing steps
sampled_df['Year-Of-Publication'] = pd.to_numeric(sampled_df['Year-Of-Publication'], errors='coerce')
sampled_df = sampled_df.dropna(subset=['Year-Of-Publication'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(sampled_df[['Year-Of-Publication', 'Book-Rating']])
hasher = FeatureHasher(n_features=100, input_type='string')
hashed_features = hasher.transform(sampled_df['Book-Author'].apply(lambda x: [x]).tolist()).toarray()
X = np.hstack((scaled_features, hashed_features))

# Range of k values to test
k_values = range(2,3)  # Test from 2 to 9 clusters

# Dictionary to store silhouette scores for each k
silhouette_scores = {}

for k in k_values:
    mbk = MiniBatchKMeans(n_clusters=k, batch_size=100, verbose=0)
    mbk_labels = mbk.fit_predict(X)
    silhouette_score_k = silhouette_score(X, mbk_labels)
    silhouette_scores[k] = silhouette_score_k
    print(f'k={k}, Silhouette Score: {silhouette_score_k}')

# Output the best k value
best_k = max(silhouette_scores, key=silhouette_scores.get)
print(f'Best k value: {best_k} with a silhouette score of {silhouette_scores[best_k]}')


C:\Users\tyler\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


k=2, Silhouette Score: 0.7636202888934444
Best k value: 2 with a silhouette score of 0.7636202888934444


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

# Function to sample a percentage of the data
def sample_data(df, percentage=50):
    return df.sample(frac=percentage/100)

# Set the percentage of data to use
data_percentage = 5  # Change this value between 0 and 100 as needed

# Sample the data
sampled_df = sample_data(cleaned_combined_df, percentage=data_percentage)

# Preprocessing steps
sampled_df['Year-Of-Publication'] = pd.to_numeric(sampled_df['Year-Of-Publication'], errors='coerce')
sampled_df = sampled_df.dropna(subset=['Year-Of-Publication'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(sampled_df[['Year-Of-Publication', 'Book-Rating']])
hasher = FeatureHasher(n_features=100, input_type='string')
hashed_features = hasher.transform(sampled_df['Book-Author'].apply(lambda x: [x]).tolist()).toarray()
X = np.hstack((scaled_features, hashed_features))

# Mean Shift clustering
ms = MeanShift(bin_seeding=True, n_jobs=-1)  # bin_seeding and n_jobs for faster computation
print("Mean Shift clustering in progress...")
ms_labels = ms.fit_predict(X)
print("Clustering complete.")

# Silhouette score
ms_silhouette = silhouette_score(X, ms_labels)
print('Mean Shift Silhouette Score:', ms_silhouette)


Mean Shift clustering in progress...
Clustering complete.
Mean Shift Silhouette Score: 0.7625203770280946


In [8]:
# Extracting the cluster labels
labels = ms.labels_

# Calculating the number of clusters
n_clusters_ = len(np.unique(labels))

print(f"Number of clusters found by Mean Shift: {n_clusters_}")

Number of clusters found by Mean Shift: 2


In [9]:
# Estimating the bandwidth on a sample of the data
bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

# Use the estimated bandwidth to fit the Mean Shift model
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)

# Extracting the new cluster labels and counting clusters
labels = ms.labels_
n_clusters_ = len(np.unique(labels))

print(f"Estimated Bandwidth: {bandwidth}")
print(f"Number of clusters found by Mean Shift after adjustment: {n_clusters_}")

Estimated Bandwidth: 1.5880321856815443
Number of clusters found by Mean Shift after adjustment: 2


In [21]:
# Function to sample a percentage of the data
def sample_data(df, percentage=50):
    return df.sample(frac=percentage/100, random_state=42)

# Set the percentage of data to use
data_percentage = 10  # Change this value between 0 and 100 as needed

# Assuming 'cleaned_combined_df' is your DataFrame
# Sample a percentage of the data
sampled_df = sample_data(cleaned_combined_df, percentage=data_percentage)

# Convert 'Year-Of-Publication' to numeric and handle possible non-numeric entries
sampled_df['Year-Of-Publication'] = pd.to_numeric(sampled_df['Year-Of-Publication'], errors='coerce')
sampled_df = sampled_df.dropna(subset=['Year-Of-Publication'])

# Scale the numeric features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(sampled_df[['Year-Of-Publication', 'Book-Rating']])

# Feature hashing for 'Book-Author'
hasher = FeatureHasher(n_features=100, input_type='string')
hashed_features = hasher.transform(sampled_df['Book-Author'].apply(lambda x: [x]).tolist()).toarray()

# Combine scaled and hashed features
X = np.hstack((scaled_features, hashed_features))

# Hierarchical Clustering
n_clusters = 2  # Adjust the number of clusters as needed
hclust = AgglomerativeClustering(n_clusters=n_clusters)
hclust_labels = hclust.fit_predict(X)

# Silhouette score
hclust_silhouette = silhouette_score(X, hclust_labels)
print('Hierarchical Clustering Silhouette Score:', hclust_silhouette)

Hierarchical Clustering Silhouette Score: 0.7597989338949945


In [15]:
import pandas as pd

# Load the datasets
books_df = pd.read_csv(r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\books_cleaned.csv', dtype={'Year-Of-Publication': str})
ratings_df = pd.read_csv(r'C:\Users\tyler\OneDrive\Desktop\GitHub Repositories\DataScience2\Exit_Tickets\March28th\CleanedData\ratings_cleaned.csv')

# Find the books that user 805 has already read
user_805_read = ratings_df[ratings_df['User-ID'] == 805]['ISBN'].unique()

# Filter these books from the main books dataset
books_to_recommend = books_df[~books_df['ISBN'].isin(user_805_read)]

# Recommend a book with the highest rating
# First, merge the books dataframe with the ratings dataframe to get average ratings
merged_books_ratings = pd.merge(books_to_recommend, ratings_df, on='ISBN')
average_ratings = merged_books_ratings.groupby('ISBN')['Book-Rating'].mean().reset_index()

# Merge back with books dataframe to get book titles
recommended_books = pd.merge(average_ratings, books_to_recommend, on='ISBN')

# Get the highest-rated book
highest_rated_book = recommended_books.sort_values('Book-Rating', ascending=False).head(1)

# Output the recommendation
if not highest_rated_book.empty:
    book_title = highest_rated_book['Book-Title'].values[0]
    print(f"Recommended Book for User 805: {book_title}")
else:
    print("No book found to recommend.")


Recommended Book for User 805: X/1999-Duet   (Book 6)
